In [1]:
import os
import json
import requests
from dotenv import load_dotenv
from pprint import pprint

# we now use OpenAI to send requests
from openai import OpenAI

In [2]:
# for console formatting I use the Rich library
from rich.console import Console

console = Console()

In [3]:
# We are familiar with a standard API request
# This is a public API that returns a random joke
# The API developers have a very deterministic response even if random
# Both sides program imperatively

get_random_joke_internet = requests.get(
    "https://official-joke-api.appspot.com/random_joke"
)
print(get_random_joke_internet.json())

{'type': 'general', 'setup': 'What does an angry pepper do?', 'punchline': 'It gets jalapeño face.', 'id': 236}


In [4]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins:{openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins:sk-proj-TV


In [5]:
client = OpenAI()
MODEL = "gpt-4o-mini"
print(f"MODEL: {MODEL}")

MODEL: gpt-4o-mini


In [6]:
# We now create our ENDPOINT but on the client side and in NATURAL LANGUAGE
system_message = """
You are an assistant that is great at telling jokes.
"""

# Here is where we can do some prompt engineering - we are adding to the system message and creating our endpoint as it were.
prompt_engineering = """
A joke worthy of publishing is a joke that has a rating of 8.5/10 or above.

If the joke is worthy of publishing also include next: PUBLISH otherwise next: RETRY

# Example

Here is an example of a joke worth of publishing:
Supply the response in the following JSON format:
{"setup": "The setup of the joke",
"punchline": "The punchline of the joke",   
"rating": "9.0",
"next": "PUBLISH"
}

Remove all back ticks and other unnecessary characters and just print the JSON format and nothing else.

Please ensure jokes are not repeated on retries

"""

# We add the prompt engineering to the system message
system_message += prompt_engineering

# We create our user prompt and will add all of these to the payload.
user_prompt = "Tell a light-hearted joke for an audience of Pythonistas"

In [7]:
# We create a list of messages to pass to the LLM

# OpenAI works was trained with a list of messages - system, user, assistant - so using this is most effective

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]
# we will get back as we have seen previously an 'assistant' message.

In [8]:
# We saw previously what is returned in the response object
# Actual implementation varies from LLM to LLM, but LiteLLM is an online service that create a uniform interface to all LLMs amongst other features and may well be of use.

response = client.chat.completions.create(model=MODEL, messages=prompts)
print(response.choices[0].message.content)
output = response.choices[0].message.content.replace("\n", "")

{"setup": "Why do Python programmers prefer dark mode?", "punchline": "Because light attracts bugs!", "rating": "8.7", "next": "PUBLISH"}


For the application, we create a state object so that we can track the state of the application - it is custom.


In [9]:
state = {"next": "", "setup": "", "punchline": "", "rating": ""}

In [10]:
print(output)

{"setup": "Why do Python programmers prefer dark mode?", "punchline": "Because light attracts bugs!", "rating": "8.7", "next": "PUBLISH"}


In [11]:
# We can update the state object for use in our app...
result = json.loads(output)
pprint(result)
if result["next"] == "PUBLISH":
    state["next"] = result["next"]
    state["setup"] = result["setup"]
    state["rating"] = result["rating"]
    state["punchline"] = result["punchline"]
else:
    state["next"] = "RETRY"

{'next': 'PUBLISH',
 'punchline': 'Because light attracts bugs!',
 'rating': '8.7',
 'setup': 'Why do Python programmers prefer dark mode?'}


In [12]:
# We can extract the NEXT step from the Autonomous AI Agent.

# This can then be used in a range of sofware design patterns for program flow.

# This illustrates the AUTONOMY aspect of AI Agents.

console.print(f"next: [dark_orange bold]{state['next']}[/]")

next: PUBLISH